# Lab 1: Set Covering

Given a number $N$ and some lists of integers $P = (L_0, L_1, L_2, ..., L_n)$, <br>
determine, if possible, $S = (L_{s_0}, L_{s_1}, L_{s_2}, ..., L_{s_n})$<br>
such that each number between $0$ and $N-1$ appears in at least one list<br>
<br>
$forall \ n \ in \ [0, N-1] \ exists \ i : n \ in \ L_{s_i}$<br>
<br>
and that the total numbers of elements in all $L_{s_i}$ is minimum.

In [1]:
import random
from gx_utils import *
from copy import deepcopy
import numpy as np
import logging
from random import seed, choice
from typing import Callable

In [2]:
def problem(N, seed=None):
    random.seed(seed)
    return [
        list(set(random.randint(0, N - 1) for n in range(random.randint(N // 5, N // 2))))
        for n in range(random.randint(N, N * 5))
    ]

In [3]:
logging.basicConfig(format="%(message)s", level=logging.INFO)

In [4]:
class State:
    def __init__(self, data: np.array):
        self._data = data.copy()
        self._data.flags.writeable = False

    def __hash__(self):
        return hash(bytes(self._data))

    def __eq__(self, other):
        return bytes(self._data) == bytes(other._data)

    def __lt__(self, other):
        return bytes(self._data) < bytes(other._data)

    def __str__(self):
        return str(self._data)

    def __repr__(self):
        return repr(self._data)

    @property
    def data(self):
        return self._data

    def copy_data(self):
        return self._data.copy()

In [5]:
def search(
    initial_state: State,
    goal_test: Callable,
    state_cost: dict,
    priority_function: Callable,
    unit_cost: Callable,
    beam = 0,
):
    frontier = PriorityQueue()
    state_cost.clear()

    state = initial_state
    state_cost[state] = 0

    while state is not None and not goal_test(state):
        count = 0
        for a in possible_actions(state):
            count +=1
            if beam and count >= beam:
                break
            new_state = result(state, a)
            cost = unit_cost(a)
            if new_state not in state_cost and new_state not in frontier:
                state_cost[new_state] = state_cost[state] + cost
                frontier.push(new_state, p=priority_function(new_state))
                logging.debug(f"Added new node to frontier (cost={state_cost[new_state]})")
            elif new_state in frontier and state_cost[new_state] > state_cost[state] + cost:
                old_cost = state_cost[new_state]
                state_cost[new_state] = state_cost[state] + cost
                logging.debug(f"Updated node cost in frontier: {old_cost} -> {state_cost[new_state]}")
        if frontier:
            state = frontier.pop()
        else:
            state = None

    if state:
        logging.info(f"Found a solution for N={N}: w={state_cost[state]:,}; visited {len(state_cost):,} nodes")
    else:
        logging.info("Solution not found!")
    
    return 1 if state else 0

In [6]:
SEED = 42
N = 1000
seed(SEED)

In [7]:
INITIAL_STATE = State(np.array([]))


GOAL = State(np.array(list(set(range(N)))))
logging.info(f"Goal:\n{GOAL}")


def goal_test(state):
    return state == GOAL

Goal:
[  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53
  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125
 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143
 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179
 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197
 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215
 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233
 234 235 236 237 238 239 240 241 242 243 244 

In [8]:
def is_valid(state: list, action: list):
    valid = not all(item in state for item in action)
    return valid


all_lists = problem(N, seed=SEED)
def possible_actions(state: State):
    return (m for m in all_lists if is_valid(list(state._data), m))


def result(state: State, action: list):
    return State(np.array(list(set(state.copy_data()).union(action))))

def h(state):
    return N - len(state)

## Breadth-First

In [9]:
state_cost = dict()

final = search(
    INITIAL_STATE,
    goal_test=goal_test,
    state_cost=state_cost,
    priority_function=lambda s: len(s._data),
    unit_cost=lambda a: len(a),
)

KeyboardInterrupt: 

## Depth-First

In [10]:
state_cost = dict()

final = search(
    INITIAL_STATE,
    goal_test=goal_test,
    state_cost=state_cost,
    priority_function=lambda s: -len(s._data),
    unit_cost=lambda a: len(a),
)

Found a solution for N=1000: w=2,893; visited 24,238 nodes


## A*

In [11]:
state_cost = dict()

final = search(
    INITIAL_STATE,
    goal_test=goal_test,
    state_cost=state_cost,
    priority_function=lambda s: len(s._data) + h(s._data),
    unit_cost=lambda a: len(a),
)

Found a solution for N=1000: w=3,717; visited 255,965 nodes
